## lambda function hands-on

In [ ]:
import boto3, botocore
from botocore.exceptions import ClientError
import os, time, json, io, zipfile, subprocess, shutil
from datetime import date
from dotenv import load_dotenv

from pathlib import Path

from misc import load_from_yaml, save_to_yaml
import iam, s3, lf, rds, vpc, ec2, lambdafn as lfn

load_dotenv(".env")
# boto3.setup_default_session(profile_name="AMominNJ")

True

In [ ]:
ACCOUNT_ID        = os.environ['AWS_ACCOUNT_ID_ROOT']
REGION            = os.environ['AWS_DEFAULT_REGION']
VPC_ID            = os.environ['AWS_DEFAULT_VPC']
SECURITY_GROUP_ID = os.environ['AWS_DEFAULT_SG_ID']
SUBNET_IDS        = SUBNET_IDS = os.environ["AWS_DEFAULT_SUBNET_IDS"].split(":")
SUBNET_ID         = SUBNET_IDS[0]
print(SUBNET_IDS)

In [ ]:
sts_client           = boto3.client('sts')
iam_client           = boto3.client('iam')
s3_client            = boto3.client('s3')
ec2_client           = boto3.client('ec2', region_name=REGION)
ec2_resource         = boto3.resource('ec2', region_name=REGION)
sfn_client           = boto3.client('stepfunctions')
logs_client          = boto3.client('logs')
events_client        = boto3.client('events')

rds_client           = boto3.client('rds')


sns_client           = boto3.client('sns')
sqs_client           = boto3.client('sqs')
lfn_client           = boto3.client('lambda')

-   [AWS Lambda Function Execution and Cold Start](https://www.youtube.com/watch?v=BhQh9QZdiKQ&list=PL9nWRykSBSFjodfc8l8M8yN0ieP94QeEL&index=30)
-   [What is Lambda Throttling? (and how to fix it!) | AWS Feature Overview]()
-   [AWS Lambda Concurrency | Reserved Concurrency | Provisioned Concurrency](https://www.youtube.com/watch?v=oanj5HKaUzs&t=567s)

#### Lambda API

In [ ]:
# [method for method in dir(lfn_client) if not method.startswith("_")]

#### Lambda Creation

In [ ]:
LFN_NAME = 'httx-lfn-nmae'
LFN_ROLE_NAME = "httx-lfn-role-name"

In [ ]:
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "lambda.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

# Create the IAM role with the assume role policy document
LFN_ROLE_ARN = iam_client.create_role(
    RoleName=LFN_ROLE_NAME,
    AssumeRolePolicyDocument=json.dumps(assume_role_policy_document)
)['Role']['Arn']

In [ ]:
# policy_arns = [
#     'arn:aws:iam::aws:policy/AmazonSQSFullAccess',
#     # 'arn:aws:iam::aws:policy/service-role/AmazonAPIGatewayPushToCloudWatchLogs',
#     # 'arn:aws:iam::aws:policy/CloudWatchFullAccess'
# ]

# [iam_client.attach_role_policy(RoleName=LFN_ROLE_NAME, PolicyArn=arn) for arn in policy_arns]

In [ ]:
# Put inline policy to enable CloudWatch logging
cloudwatch_logs_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "logs:CreateLogGroup",
                "logs:CreateLogStream",
                "logs:PutLogEvents"
            ],
            "Resource": "arn:aws:logs:*:*:*"
        }
    ]
}

# Attach the CloudWatch Logs policy to the role
response = iam_client.put_role_policy(
    RoleName=LFN_ROLE_NAME,
    PolicyName='CloudWatchLogsPolicy',
    PolicyDocument=json.dumps(cloudwatch_logs_policy)
)

In [69]:
# create_lambda_package("./lambdas", "./", py_lib=["PyMySQL"])

In [ ]:
LFN_ARN = lfn.create_lambda_function(LFN_NAME, LFN_ROLE_ARN, 'handlers.sqs_processor', './package.zip')

##### Analysis

In [ ]:
# response = lfn_client.add_permission(
#     FunctionName=LFN_NAME,       # Replace with your Lambda function name
#     StatementId='',  # An identifier for this statement, unique for each permission you add
#     Action='lambda:InvokeFunction',
#     Principal='',
#     SourceArn="",  # Replace with your S3 bucket ARN
#     SourceAccount=ACCOUNT_ID  # Your AWS account ID
# )

# print("Lambda permission added:", response)


In [ ]:
payload = {
    "detail-type": "Glue Crawler State Change",
    "detail": {"crawlerName": "httx-s3_clensed_crawler",}
}

response = lfn_client.invoke(
    FunctionName=LFN_NAME,
    InvocationType='RequestResponse',  # 'RequestResponse' for synchronous execution
    Payload=json.dumps(payload)
)

# Read the response
response_payload = json.loads(response['Payload'].read())
print("Response:")
print(json.dumps(response_payload, indent=4))

In [ ]:
lambdafn.print_latest_lambda_logs(LFN_NAME)

In [ ]:
lfn_client.delete_function(FunctionName=LFN_NAME)

#### S3 + Lambda

In [ ]:
S3_BUCKET_DATALAKE = "httx-bkt"
S3_BUCKET_GLUE_ASSETS = "httx-glue-assets-bkt"

In [ ]:
acl = 'public-read'                         # Set the ACL (e.g., 'private', 'public-read')
enable_versioning = False                   # Enable versioning
enable_encryption = False                   # Enable server-side encryption

folders1 = ['raw/customers/', 'processed/customers/']
folders2 = ['temporary/', 'sparkHistoryLogs/']

s3.create_s3_bucket(S3_BUCKET_DATALAKE, folders1)
s3.create_s3_bucket(S3_BUCKET_GLUE_ASSETS, folders2)

In [ ]:
# Add S3 trigger to the Lambda function
response = s3_client.put_bucket_notification_configuration(
    Bucket=S3_BUCKET_DATALAKE,
    NotificationConfiguration={
        'LambdaFunctionConfigurations': [
            {
                'LambdaFunctionArn': LFN_ARN,
                'Events': [
                    's3:ObjectCreated:*'  # Trigger Lambda on object creation
                ],
                'Filter': {
                    'Key': {
                        'FilterRules': [
                            {
                                'Name': 'prefix',
                                'Value': 'raw/customers/'  # Trigger only on this prefix
                            },
                        ]
                    }
                }
            }
        ]
    }
)

print("S3 bucket notification configuration updated successfully.")


#### SQS + Lambda

-   [AWS SQS to Lambda Tutorial in NodeJS | Step by Step](https://www.youtube.com/watch?v=JJQrVBRzlPg&t=762s)
-   [AWS SQS + Lambda Setup Tutorial - Step by Step](https://www.youtube.com/watch?v=xyHLX1dUwuA)
-   [Lambda + SQS Users Should Know About This](https://www.youtube.com/watch?v=0707Py8Jyf0)

In [ ]:
# iam_client.get_role(RoleName=LFN_ROLE_NAME)

In [ ]:
# iam_client.attach_role_policy(RoleName=LFN_ROLE_NAME, PolicyArn='arn:aws:iam::aws:policy/AmazonSQSFullAccess')
# iam_client.detach_role_policy(RoleName=LFN_ROLE_NAME, PolicyArn='arn:aws:iam::aws:policy/AmazonSQSFullAccess')

In [ ]:
QUE_NAME = 'httx_sqs'

# Define attributes dictionary for configuration
attributes = {
    # Maximum message size (default 262144 bytes or 256 KB, max 262144)
    'MaximumMessageSize': '262144',
    
    # Message retention period (default 345600 seconds or 4 days, max 1209600)
    'MessageRetentionPeriod': '86400',  # 1 day
    
    # Visibility timeout (default 30 seconds, max 43200 or 12 hours)
    'VisibilityTimeout': '60',
    
    # Delivery delay (default 0 seconds, max 900 seconds or 15 minutes)
    'DelaySeconds': '0',
    
    # Enable content-based deduplication for FIFO queues only
    # Uncomment if using a FIFO queue
    # 'ContentBasedDeduplication': 'true',
    
    # Set queue type to FIFO (requires .fifo suffix for QueueName)
    # Uncomment if using a FIFO queue
    # 'FifoQueue': 'true',

    # Enable server-side encryption with KMS
    # 'KmsMasterKeyId': 'alias/aws/sqs',  # Use AWS managed key, or specify your own KMS Key ARN
    # 'KmsDataKeyReusePeriodSeconds': '300',  # 5 minutes reuse period for data encryption keys
    
    # Configure dead-letter queue (DLQ) if needed
    # Replace 'DeadLetterQueueARN' with your DLQ ARN
    # Uncomment and specify the DLQ ARN to use this option
    # 'RedrivePolicy': '{"maxReceiveCount":"5", "deadLetterTargetArn":"DeadLetterQueueARN"}'
}


In [ ]:
# Create the SQS queue with the specified attributes
queue_url = sqs_client.create_queue(
    QueueName=QUE_NAME,
    Attributes=attributes
)['QueueUrl']

print(queue_url)

In [ ]:
# que_url = sqs_client.get_queue_url(QueueName=QUE_NAME)['QueueUrl']

In [ ]:
# Get the SQS queue ARN
# queue_url = sqs_client.get_queue_url(QueueName=QUE_NAME)['QueueUrl']
queue_arn = sqs_client.get_queue_attributes(QueueUrl=queue_url, AttributeNames=['QueueArn'])['Attributes']['QueueArn']

# Create event source mapping for SQS
event_source_mapping_id = lfn_client.create_event_source_mapping(
    EventSourceArn=queue_arn,
    FunctionName=LFN_NAME,
    BatchSize=10  # Adjust batch size as needed
)['UUID']

print("SQS trigger added to Lambda function:", response['UUID'])


In [ ]:
message_body = 'Test-Message-2'
message_attributes = {}
response = sqs_client.send_message(
            QueueUrl=queue_url,
            MessageBody=message_body,
            MessageAttributes=message_attributes or {}
        )
message_id = response['MessageId']

In [ ]:
response = sqs_client.receive_message(
            QueueUrl=queue_url,
            MaxNumberOfMessages=10,
            WaitTimeSeconds=10
        )
messages = response.get('Messages', [])

len(messages)

In [ ]:
# Delete messages
for message in messages:
    print(message['Body'])
    # sqs_client.delete_message(
    #     QueueUrl=que_url,
    #     ReceiptHandle=message['ReceiptHandle']
    # )

In [ ]:
sqs_client.delete_queue(QueueUrl=queue_url)

In [ ]:
lfn_client.delete_event_source_mapping(UUID=event_source_mapping_id)

#### SNS + Lambda

In [ ]:
TOPIC_NAME = 'httx-sns-topic'

In [ ]:
JOB_COMPLETE_TOPIC_ARN = sns_client.create_topic(Name=TOPIC_NAME)['TopicArn']

protocol="email"
endpoint="bbcredcap3@gmail.com"

sns_client.subscribe(
    TopicArn=JOB_COMPLETE_TOPIC_ARN,
    Protocol=protocol,
    Endpoint=endpoint
)

## [ETL From AWS S3 to Amazon Redshift with AWS Lambda dynamically.](https://www.youtube.com/watch?v=JyQ9EFFR3n8&list=PLO95rE9ahzRsdzmZ_ZT-3uOn1Nh2eEpWB&index=26&t=1048s)

-   [lab](https://github.com/RekhuGopal/PythonHacks/tree/main/AWSBoto3Hacks/AWS-ETL-S3-Lambda-Redshift)

## [AWS Tutorials - Programming to Access Amazon RDS using AWS Lambda](https://www.youtube.com/watch?v=4sqsyDJ2Kh0)

### Images

<div style="text-align:center"><img src="./images/screenshot.png" height="250p" height="200p"></img</dev>

<div style="text-align:center"><img src="./images/screenshot1.png" height="270p" height="200p"></img</dev>

<div style="text-align:center"><img src="./images/screenshot2.png" height="260p" height="200p"></img</dev>

<div style="text-align:center"><img src="./images/screenshot3.png" height="260p" height="200p"></img</dev>

#### Create RDS

In [ ]:
DB_NAME = 'Employee_DB'
DB_USERNAME = os.environ['USERNAME']
DB_PASSWORD = os.environ['PASSWORD']

In [ ]:
SUBNET_GROUP_NAME = 'httx-rds-subnet-group'
## Create the RDS subnet group
response = rds_client.create_db_subnet_group(
    DBSubnetGroupName=SUBNET_GROUP_NAME,
    DBSubnetGroupDescription='Subnet group for RDS instance',
    SubnetIds=SUBNET_IDS
)
print(response)

In [ ]:
instances = [
    {
        'db_instance_identifier': 'httx-rds-mysql',
        'db_name': DB_NAME,
        'db_username': DB_USERNAME,
        'db_password': DB_PASSWORD,
        'engine': 'mysql',
        'port': 3306,
        'engine_version': '8.0.32',
        'db_instance_class': 'db.t3.micro',
        'allocated_storage': 20,
        'availability_zone': 'us-east-1a',
        'tags': [{'Key': 'Project', 'Value': 'glue-rds-Crawler'}],
        'security_group_ids': [SECURITY_GROUP_ID],
        'db_subnet_group_name': SUBNET_GROUP_NAME,
    },
    {
        'db_instance_identifier': 'httx-rds-postgresql',
        'db_name': DB_NAME,
        'db_username': DB_USERNAME,
        'db_password': DB_PASSWORD,
        'port': 5432,
        'engine': 'postgres',
        'engine_version': '14.13',
        'db_instance_class': 'db.t3.micro',
        'allocated_storage': 20,
        'availability_zone': 'us-east-1a',
        'tags': [{'Key': 'Project', 'Value': 'glue-rds-Crawler'}],
        'security_group_ids': [SECURITY_GROUP_ID],
        'db_subnet_group_name': SUBNET_GROUP_NAME,
    },
    {
        'db_instance_identifier': 'httx-rds-mssql',
        'db_name': '',
        'db_username': DB_USERNAME,
        'db_password': DB_PASSWORD,
        'port': 1433,
        'engine': 'sqlserver-ex',
        'engine_version': '15.00.4153.1.v1',
        'db_instance_class': 'db.t3.micro',
        'allocated_storage': 20,
        'availability_zone': 'us-east-1a',
        'tags': [{'Key': 'Project', 'Value': 'glue-rds-Crawler'}],
        'security_group_ids': [SECURITY_GROUP_ID],
        'db_subnet_group_name': SUBNET_GROUP_NAME,
    },
]

In [ ]:
rds.create_rds_instance(**instances[0])   # 'httx-rds-mysql'

In [ ]:
# Describe the RDS instance
response = rds_client.describe_db_instances(
    DBInstanceIdentifier=instances[0]['db_instance_identifier']
)

# Extract the instance details
db_instances = response['DBInstances']
if db_instances:
    instance = db_instances[0]
    status = instance['DBInstanceStatus']
    
    if status == 'available':
        mysql_endpoint = instance['Endpoint']['Address']
        print(f"RDS Endpoint: {mysql_endpoint}")
    else:
        print(f"RDS instance is in {status} state, NO ENDPOINT AVAILABLE YET!!")
else:
    print("No RDS instance found.")

-   `Gateway` endpoints serve as a target for a route in your route table for traffic destined for the service.

In [ ]:
# VPC Endpoint parameters
VPC_ENDPOINT_TAG = 'rds-glue-vpc-endpoint'
VPC_ENDPOINT_SERVICE_NAME = 'com.amazonaws.us-east-1.s3'
SECURITY_GROUP_IDS = [SECURITY_GROUP_ID]  # Security group(s) associated with the endpoint
ROUTE_TABLE_IDS = ['rtb-0ec4311296ec952f8']

# Create an Interface Endpoint
VPC_ENDPOINT_ID = ec2_client.create_vpc_endpoint(
    VpcEndpointType='Gateway',
    VpcId=VPC_ID,
    ServiceName=VPC_ENDPOINT_SERVICE_NAME,
    RouteTableIds=ROUTE_TABLE_IDS,
    # SubnetIds=sg_id,
    # SecurityGroupIds=security_group_ids,
    PrivateDnsEnabled=False  # Enable private DNS to resolve service names within the VPC
)['VpcEndpoint']['VpcEndpointId']

In [ ]:
ec2_client.create_tags(Resources=[VPC_ENDPOINT_ID],Tags=[{'Key': 'Name', 'Value': 'rds_vpc_endpoint'}])

###### Load sql data from Local Machine to RDS Instance

-   Load into MySQL (TESTED):

    -   `$ mysql -h <rds-endpoint> -p <port> -U <username> -d <dbname>` -> Connect via Command Line if needed
    -   `$ mysql -h {mysql_endpoint} -P {mysql_port} -u httxadmin -p'{DB_PASSWORD}' interview_questions < /Users/am/mydocs/Software_Development/Web_Development/aws/aws_rds/interview_questions.sql`

In [ ]:
# ! mysql -h {mysql_endpoint} -P {instances[0]['port']} -u {DB_USERNAME} -p'{DB_PASSWORD}' {DB_NAME} < ./mysql_employees.sql

### Part-1

- Zip the lambda Function with dependencies (e.g. `pymysql`)
- Create Lambda Function

```python
Using pymysql module
====================

import json
import pymysql

def lambda_handler(event, context):
    conn = pymysql.connect(host='', user='', database='', password='', cursorclass=pymysql.cursors.DictCursor)
    with conn.cursor() as cur:
        cur.execute("insert into myfriends values ('firstname1','lastname1')")
        conn.commit()
        cur.close()
        conn.close()
    
    # TODO implement
    return {
        'statusCode': 200,
        'body': json.dumps('data inserted')
    }
```


In [ ]:
# create_lambda_package("./lambdas", "./", py_lib=["PyMySQL"])

In [ ]:
LFN_ARN = create_lambda_function(LFN_NAME, LFN_ROLE_ARN, 'handlers.get_from_rds', './package.zip')

### Part-2

- Zip the lambda Function
- Create Lambda Function
- Create a AWS Layer of `awswrangler`
- Add the layer to Lambda Function


```python

Using awswrangler module
========================

import json
import awswrangler as wr

def lambda_handler(event, context):
    conn = wr.mysql.connect("dojoconnection")
    with conn.cursor() as cur:
        cur.execute("insert into myfriends values ('firstname1','lastname1')")
        conn.commit()
        cur.close()
        conn.close()
    
    # TODO implement
    return {
        'statusCode': 200,
        'body': json.dumps('data inserted')
    }
```

In [ ]:
# Read the .zip file as binary
with open(zip_file_path, 'rb') as zip_file:
    layer_zip_content = zip_file.read()

# Create the Lambda layer
response = lambda_client.create_layer_version(
    LayerName=layer_name,
    Description=description,
    Content={'ZipFile': layer_zip_content},
    CompatibleRuntimes=compatible_runtimes
)

In [ ]:
response = lambda_client.update_function_configuration(
    FunctionName=function_name,
    Layers=new_layers
)